Prompt: i want to keep up to date on the newest research in trauma therapy, ptsd and cptsd. I believe that searching pubmed is one way to do this, but I would like to write a script to automate it so I don't have to rewrite the queries every time. I would like to search once a month, and then get a list of the 5 most relevant/popular articles about that topic. is it possible to write a python program to do this? do they have an api? or do you recommend some other way

Yes, it’s absolutely possible to automate this with Python using the **NCBI Entrez API**, which PubMed provides via their [E-utilities](https://www.ncbi.nlm.nih.gov/books/NBK25501/). You can use the `Bio.Entrez` module from the Biopython library to query PubMed and retrieve metadata like article titles, authors, and links.

Here's a basic example script to:

* Query PubMed for trauma therapy, PTSD, and CPTSD.
* Retrieve metadata for the top 5 most relevant or recent articles.
* Output titles and links.

### Step 1: Install Biopython

```bash
pip install biopython
```

### Step 2: Example Script




In [ ]:
from Bio import Entrez
from datetime import datetime

# Set your email (required by NCBI)
Entrez.email = "your_email@example.com"

def search_pubmed(query, max_results=5):
    handle = Entrez.esearch(
        db="pubmed",
        term=query,
        retmax=max_results,
        sort="relevance",
        datetype="pdat",  # use publication date
        reldate=30        # last 30 days
    )
    record = Entrez.read(handle)
    handle.close()
    return record['IdList']

def fetch_details(id_list):
    ids = ",".join(id_list)
    handle = Entrez.efetch(db="pubmed", id=ids, retmode="xml")
    records = Entrez.read(handle)
    handle.close()
    return records

def format_results(records):
    for i, article in enumerate(records['PubmedArticle']):
        title = article['MedlineCitation']['Article']['ArticleTitle']
        authors = article['MedlineCitation']['Article'].get('AuthorList', [])
        pub_date = article['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']
        article_id = article['MedlineCitation']['PMID']
        link = f"https://pubmed.ncbi.nlm.nih.gov/{article_id}/"

        author_names = ', '.join(
            [f"{a['ForeName']} {a['LastName']}" for a in authors if 'ForeName' in a and 'LastName' in a]
        )
        print(f"{i+1}. {title}")
        print(f"   Authors: {author_names}")
        print(f"   Published: {pub_date}")
        print(f"   Link: {link}\n")

if __name__ == "__main__":
    query = "trauma therapy OR PTSD OR CPTSD"
    ids = search_pubmed(query)
    if ids:
        articles = fetch_details(ids)
        format_results(articles)
    else:
        print("No new articles found.")


### Features to Extend

* Save results to a CSV or HTML report.
* Schedule it monthly using `cron` (Linux/macOS) or Task Scheduler (Windows).
* Add email delivery via `smtplib`.

Would you like help turning this into a monthly automation or outputting results into a file or email?

# Updated script to exclude results already seen